In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from datetime import datetime

In [2]:
def load_and_normalize_data(filepath, last_n_days=30):
    df = pd.read_csv(filepath)
    close_prices = df[['Close']].values
    # print(len(close_prices))
    # print(close_prices.shape)
    # Normalize the data
    # Exclude the last 30 days for the training set
    training_data = close_prices[:-last_n_days]
    testing_data = close_prices[-last_n_days:]
    return torch.FloatTensor(training_data), torch.FloatTensor(testing_data), torch.FloatTensor(close_prices)

In [3]:
t, v, df = load_and_normalize_data('EURUSD=X(1).csv', last_n_days=30)
print(len(t))
print(len(v))
print(len(df))

1276
30
1306


In [4]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L - tw - 1):
        train_seq = input_data[i:i + tw]
        train_label = input_data[i + tw:i + tw + 1]
        inout_seq.append((train_seq, train_label))
        # print(inout_seq.shape)
    return inout_seq

In [5]:
class LSTM_GRU_Model(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=64, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        # LSTM Layer
        self.init_lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.lstm = nn.LSTM(hidden_layer_size, hidden_layer_size, batch_first=True)

        # GRU Layer - input size matches LSTM output size (hidden_layer_size)
        self.gru = nn.GRU(hidden_layer_size, hidden_layer_size, batch_first=True)
        self.s_gru = nn.GRU(hidden_layer_size // 2, hidden_layer_size // 4, batch_first=True)
        self.s2_gru = nn.GRU(hidden_layer_size // 4, hidden_layer_size, batch_first=True)
        self.gru_after_bi = nn.GRU(hidden_layer_size * 2, hidden_layer_size, batch_first=True)

        # Linear layer to get to the output size
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        # LSTM layer
        lstm_out, _ = self.init_lstm(input_seq)
        # lstm_out, _ = self.lstm(lstm_out)
        # lstm_out, _ = self.lstm(lstm_out)
        # lstm_out, _ = self.lstm(lstm_out)

        # GRU layer
        gru_out, _ = self.gru(lstm_out)
        # lstm_out, _ = self.lstm(gru_out)
        # gru_out, _ = self.s_gru(gru_out)
        # gru_out, _ = self.s2_gru(gru_out)
        # gru_out, _ = self.gru(gru_out)
        # gru_out, _ = self.gru(gru_out)

        # We use the output of the GRU layer for our predictions
        predictions = self.linear(gru_out[:, -1, :])
        return predictions

In [6]:
def train_val_model(model, train_loader, test_loader, loss_function, optimizer, epochs=10):
    for i in range(epochs):
        # train loop
        train_loss_list = []
        train_running_loss = 0.0
        s0 = datetime.now()
        for seq, labels in train_loader:
            optimizer.zero_grad()
            print(seq)
            print(labels)
            exit()

            y_pred = model(seq)        
            labels = labels.view(-1, 1)

            single_loss = loss_function(y_pred, labels)
            train_running_loss += single_loss.item() * seq.size(0)

            single_loss.backward()
            optimizer.step()

        # Calculate train loss
        train_loss = train_running_loss / len(train_loader)
        train_loss_list.append(train_loss)

        # test loop
        val_loss_list = []
        val_running_loss = 0.0
        model.eval()
        test_predictions = []
        actual_prices = []
        with torch.no_grad():
            for seq, labels in test_loader:
                predict_price = model(seq)
                labels = labels.view(-1, 1)

                test_predictions.append(predict_price)
                actual_prices.append(labels)

                single_loss = loss_function(predict_price, labels)
                val_running_loss += single_loss.item()
        print(val_running_loss)
        print(len(test_loader))
        val_loss = val_running_loss / len(test_loader[0])
        val_loss_list.append(val_loss)

        timing = datetime.now() - s0
        print(f'Epoch {i} | train_loss: {train_loss} | val_loss: {val_loss} | time: {timing}')


In [7]:

def plot_predictions(full_data, predict_data, last_n_days=30):
    # Plot the full dataset
    plt.figure(figsize=(14, 7))
    plt.plot(df.index, df['Close'], label='Actual Prices', color='grey', alpha=0.5)

    # Highlight the last N days in the dataset for actual prices
    plt.plot(df.index[-last_n_days:], df['Close'][-last_n_days:], label='Last N Days Actual Prices', color='blue')

    # Assuming predictions_scaled is a numpy array with the shape (last_n_days, 1)
    # And the last N days are the target of these predictions
    # Create a range for the last N days
    prediction_dates = df.index[-last_n_days:]
    plt.plot(prediction_dates, predictions_scaled, label='Predicted Prices', linestyle='--', color='red')

    plt.title('Bitcoin Price Prediction - Full Year with Last N Days Highlighted')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()


In [8]:
filepath = 'EURUSD=X(1).csv'
seq_length = 90
train_data, val_data, df = load_and_normalize_data(filepath, last_n_days=200)
train_sequences = create_inout_sequences(train_data, seq_length)
val_sequences = create_inout_sequences(val_data, seq_length)
train_loader = DataLoader(train_sequences, batch_size=200)
for seq, labels in train_loader:
    print('seq: ', seq.shape)
    print('labels: ',labels.shape)
    exit()
val_loader = DataLoader(val_sequences, batch_size=64)
model = LSTM_GRU_Model(hidden_layer_size=64)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_val_model(model, train_loader,val_loader, loss_function, optimizer, epochs=10)


seq:  torch.Size([200, 90, 1])
labels:  torch.Size([200, 1, 1])
seq:  torch.Size([200, 90, 1])
labels:  torch.Size([200, 1, 1])
seq:  torch.Size([200, 90, 1])
labels:  torch.Size([200, 1, 1])
seq:  torch.Size([200, 90, 1])
labels:  torch.Size([200, 1, 1])
seq:  torch.Size([200, 90, 1])
labels:  torch.Size([200, 1, 1])
seq:  torch.Size([15, 90, 1])
labels:  torch.Size([15, 1, 1])
tensor([[[1.1373],
         [1.1376],
         [1.1337],
         ...,
         [1.1287],
         [1.1293],
         [1.1288]],

        [[1.1376],
         [1.1337],
         [1.1307],
         ...,
         [1.1293],
         [1.1288],
         [1.1286]],

        [[1.1337],
         [1.1307],
         [1.1314],
         ...,
         [1.1288],
         [1.1286],
         [1.1227]],

        ...,

        [[1.1079],
         [1.1085],
         [1.1083],
         ...,
         [1.0954],
         [1.0847],
         [1.0807]],

        [[1.1085],
         [1.1083],
         [1.1105],
         ...,
         [1.0

TypeError: 'DataLoader' object is not subscriptable

In [ ]:
len(val_loader)